<a href="https://colab.research.google.com/github/rocifani/Simulacion-TPs/blob/main/TP3.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpy


In [ ]:
# Inventario

import random
import simpy

# Parámetros del sistema
tasa_arribo = 0.45
nivel_minimo_inventario = 15
nivel_maximo_inventario = 30
tiempo_demora_min = 0.5
tiempo_demora_max = 1.0
costo_faltante=5


# Variables de rendimiento
inventario_actual = 20
tot_pedidos=0
num_pedidos_entregados = 0
tiempo_total_espera = 0
tiempos_eventos = []  # Lista para almacenar los tiempos de los eventos
costo_pedido = 0
faltante=0
inventario=0
sum=0

# Probabilidades de demanda para 1, 2, 3 y 4 artículos
probabilidades_demanda = [1/6, 1/3, 1/3, 1/6]
demanda_articulos = [1, 2, 3, 4]

def generar_demanda():
    demanda = random.choices(demanda_articulos, probabilidades_demanda)[0]
    return demanda

def proceso_inventario(env):
    global inventario_actual, num_pedidos_entregados, tiempo_total_espera, tiempos_eventos, costo_pedido, tot_pedidos, faltante,inventario, sum

    pedido_pendiente = False  # Variable para controlar si hay un pedido pendiente
    tiempo_entrega = 0  # Tiempo de entrega actual del pedido
    n=1


    while env.now <= 720:

        tiempo_arribo = random.expovariate(tasa_arribo)

        inventario+=inventario_actual
        sum+=1

        # Avanzar en el tiempo (considerando el tiempo de entrega del pedido)
        yield env.timeout(tiempo_arribo)

        # Registrar tiempo de llegada de cliente
        tiempos_eventos.append(env.now)

        # Generar demanda
        demanda = generar_demanda()

        # Verificar si es necesario realizar un pedido
        if inventario_actual <= nivel_minimo_inventario and not pedido_pendiente and env.now>=30*n :
            # Realizar pedido
            num_pedidos_entregados += 1
            pedido_pendiente = True  # Hay un pedido pendiente
            tiempo_entrega = (random.uniform(tiempo_demora_min, tiempo_demora_max)) + env.now  # Calcular tiempo de entrega del pedido
            cant_ult_pedido = nivel_maximo_inventario - inventario_actual
            tiempos_eventos.append(env.now)  # Registrar tiempo en el que se realiza el pedido
            costo_pedido = costo_pedido + (32 + 3*cant_ult_pedido)
            n+=1
            print(f"{env.now:.2f}: Realizando pedido al proveedor (Inventario: {inventario_actual}), Tiempo de entrega: {tiempo_entrega:.2f}")

        # Verificar si ha llegado el pedido
        if pedido_pendiente and env.now >= tiempo_entrega:
            inventario_actual = inventario_actual + cant_ult_pedido # Actualizar el inventario con la cantidad del pedido
            pedido_pendiente = False  # El pedido ha sido entregado
            tiempos_eventos.append(env.now)  # Registrar tiempo en el que llega el pedido

        # Verificar si hay suficiente inventario para satisfacer la demanda
        if demanda <= inventario_actual:
            inventario_actual -= demanda
            print(f"{env.now:.2f}: Llegada de cliente: -{demanda} unidades (Inventario: {inventario_actual})")
        else:
            print(f"{env.now:.2f}: No hay suficientes productos disponibles.")
            faltante+=demanda

            if not pedido_pendiente:
                # No hay pedido pendiente, atender al cliente sin esperar
                print(f"{env.now:.2f}: Atendiendo al cliente sin esperar entrega de pedido...")
                inventario_actual -= demanda
                print(f"{env.now:.2f}: Llegada de cliente: -{demanda} unidades (Inventario: {inventario_actual})")
            else:
                # Hay un pedido pendiente, atender al cliente sin esperar entrega de pedido
                print(f"{env.now:.2f}: Atendiendo al cliente sin esperar entrega de pedido...")
                inventario_actual -= demanda
                print(f"{env.now:.2f}: Llegada de cliente: -{demanda} unidades (Inventario: {inventario_actual})")

# Configuración y ejecución de la simulación
env = simpy.Environment()
env.process(proceso_inventario(env))
env.run()

# Resultados
print("\nResultados de la simulación:")
print(f"Número de pedidos entregados: {num_pedidos_entregados}")
print(f"Inventario final: {inventario_actual}")
print(f"Costo de promedio de pedido: {costo_pedido/num_pedidos_entregados}")
print(f"Costo de promedio de faltante: {costo_faltante*faltante/num_pedidos_entregados}")
print(f"Costo de promedio de mantenimiento: {inventario/sum}")
print(f"Costo total:{(costo_pedido/num_pedidos_entregados)+(costo_faltante*faltante/num_pedidos_entregados)+(inventario/sum)}")

In [ ]:
# MM1

import random
import simpy
import matplotlib.pyplot as plt

# Parámetros del sistema
tiempo_simulacion = 500000  # Tiempo total de simulación
tasa_llegada = 0.66 # Parámetro lambda de la distribución exponencial para la tasa de llegada
tasa_servicio = 0.7  # Parámetro mu de la distribución exponencial para la tasa de servicio

# Variables de rendimiento
num_clientes_atendidos = 0
num_denegaciones_servicio = 0
tiempo_total_sistema = 0
tiempo_total_cola = 0
tiempo_total_servicio = 0
num_clientes_en_cola = 0
tamano_cola = []

# Variables para el gráfico de evolución de la cola de espera
tiempos = []  # Lista para almacenar los momentos en los que cambia la cola
cola_espera = []  # Lista para almacenar el tamaño de la cola en cada momento

# Variables para el gráfico del estado del servidor
estados_servidor = []  # Lista para almacenar el estado del servidor (1: ocupado, 0: libre)

def cliente(env, nombre, servidor):
    llegada = env.now
    print(f'{nombre} llegó al sistema en el tiempo {llegada:.2f}')

    global num_clientes_en_cola, tiempos, cola_espera, tamano_cola, estados_servidor  # Declarar las variables globales

    with servidor.request() as req:
        yield req

        espera = env.now - llegada
        print(f'{nombre} comenzó a ser atendido en el tiempo {env.now:.2f} (espera: {espera:.2f})')

        servicio = random.expovariate(tasa_servicio)
        yield env.timeout(servicio)

        tiempo_salida = env.now
        print(f'{nombre} se fue en el tiempo {tiempo_salida:.2f} (servicio: {servicio:.2f})')

        global num_clientes_atendidos, tiempo_total_sistema, tiempo_total_cola, tiempo_total_servicio
        num_clientes_atendidos += 1
        tiempo_total_sistema += tiempo_salida - llegada
        tiempo_total_cola += espera
        tiempo_total_servicio += servicio
        num_clientes_en_cola -= 1

        tamano_cola.append(num_clientes_en_cola)  # Almacena el tamaño de la cola actual

        tiempos.append(env.now)  # Almacena el momento en que disminuye o aumenta la cola
        cola_espera.append(max(0, num_clientes_en_cola))  # Almacena el tamaño actual de la cola (mínimo 0)

        estados_servidor.append(0 if servidor.count == 0 else 1)  # Almacena el estado actual del servidor

# Configuración y ejecución de la simulación
env = simpy.Environment()
servidor = simpy.Resource(env, capacity=1)
cola_clientes = simpy.Resource(env, capacity=1)  # Cola de clientes

def llegada_clientes(env, servidor, cola_clientes):
    global num_clientes_en_cola, tiempos, cola_espera, tamano_cola, estados_servidor  # Declarar las variables globales
    i = 0
    while True:
        yield env.timeout(random.expovariate(tasa_llegada))
        i += 1
        env.process(cliente(env, f'Cliente {i}', servidor))
        num_clientes_en_cola += 1
        if i == 1:
            tiempos.append(env.now)  # Almacena el momento en que comienza el primer cliente
            cola_espera.append(0)  # La cola comienza en cero
        else:
            tiempos.append(env.now)  # Almacena el momento en que disminuye o aumenta la cola
            cola_espera.append(max(0, num_clientes_en_cola - 1))  # Almacena el tamaño actual de la cola (mínimo 0)
        estados_servidor.append(0 if servidor.count == 0 else 1)  # Almacena el estado actual del servidor

env.process(llegada_clientes(env, servidor, cola_clientes))
env.run(until=tiempo_simulacion)

# Resultados
tiempo_promedio_sistema = tiempo_total_sistema / num_clientes_atendidos
tiempo_promedio_cola = tiempo_total_cola / num_clientes_atendidos
tiempo_promedio_servicio = tiempo_total_servicio / num_clientes_atendidos
promedio_clientes_sistema = tiempo_total_sistema / tiempo_simulacion
promedio_clientes_cola = tiempo_total_cola / tiempo_simulacion
utilizacion_servidor = (tiempo_total_servicio / tiempo_simulacion) * 100

# Gráfico de la evolución de la cola de espera
plt.figure(figsize=(10, 6))
plt.step(tiempos, cola_espera, where='post')
plt.xlabel('Tiempo')
plt.ylabel('Clientes en cola de espera')
plt.title('Evolución de la cola de espera a lo largo del tiempo')
plt.grid(True)
plt.show()

# Histograma del estado del servidor
plt.figure(figsize=(10, 4))
plt.step(tiempos, estados_servidor, where='post')
plt.xlabel('Tiempo')
plt.ylabel('Estado del servidor')
plt.title('Estado del servidor a lo largo del tiempo')
plt.ylim(-0.1, 1.1)
plt.grid(True)
plt.show()

# Probabilidad de encontrar N clientes en cola (ingresado por teclado)
n = 2
probabilidad_n_clientes = cola_espera.count(n) / len(cola_espera)

# Resultados
probabilidad_denegacion_servicio = num_denegaciones_servicio / num_clientes_atendidos

print('\nResultados de la simulación:')
print(f'Número de clientes atendidos: {num_clientes_atendidos}')
print(f'Tiempo promedio en el sistema: {tiempo_promedio_sistema:.2f}')
print(f'Tiempo promedio en la cola: {tiempo_promedio_cola:.2f}')
print(f'Tiempo promedio de servicio: {tiempo_promedio_servicio:.2f}')
print(f'Promedio de clientes en el sistema: {promedio_clientes_sistema:.2f}')
print(f'Promedio de clientes en la cola: {promedio_clientes_cola:.2f}')
print(f'Porcentaje de utilización del servidor: {utilizacion_servidor:.2f}%')
print(f'Probabilidad de encontrar {n} clientes en cola: {probabilidad_n_clientes:.4f}')
print(f'Probabilidad de denegación de servicio: {probabilidad_denegacion_servicio:.4f}')